## Goals

The goals of the coding part of this homework assignment are to:
 * Practice specifying and fitting linear regression and multinomial regression models in Keras
 * See how you can do the calculations to generate predictions from these models directly in numpy

## Module Imports
Please add code below to import numpy and any keras submodules you need.  Set a seed for random number generation from numpy before importing keras.  I've already imported pandas.

In [1]:
import pandas as pd
import numpy as np
np.random.seed(987442)
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras import optimizers
from scipy.special import softmax

Using TensorFlow backend.


## Problem 1: House sale price prediction
We have a data set with a number of characteristics of houses sold in the city of Ames, IA, as well as the sale price of the house.  Let's fit a model to predict sales price of a house.  (Original data source: De Cock, D. (2011). Journal of Statistics Education, Volume 19, Number 3.)


In [2]:
# read in data
house_prices = pd.read_csv("http://www.evanlray.com/data/AmesHousing/AmesHousing.csv")
house_prices = house_prices[['Overall_Qual', 'Overall_Cond', 'Lot_Area',
  'Bldg_Type', 'Street', 'Total_Bsmt_SF', 'Heating_QC', 'Gr_Liv_Area',
  'Bsmt_Full_Bath', 'Fireplaces', 'Garage_Cars', 'Garage_Area', 'Wood_Deck_SF',
  'Year_Built', 'Year_Remod_Add', 'Sale_Price']]
house_prices = pd.get_dummies(house_prices, drop_first = True)

# how much data do we have?
print("shape of house_prices = " + str(house_prices.shape))

shape of house_prices = (2930, 37)


In [0]:
X = house_prices.drop(['Sale_Price'], axis=1).to_numpy()
y = house_prices['Sale_Price'].to_numpy().reshape((house_prices.shape[0], 1))

In [12]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(1875, 36)
(586, 36)
(469, 36)


#### (a) Define an appropriate model in Keras.  Use Adam ('adam') for your optimizer and no additional metrics of performance other than your loss function.  Run the estimation process for about 1000 epochs using a batch size equal to the number of examples in your training set (this will take about a minute).  We'll see what these things mean starting next week.
Things to consider:

* How many units do you need in your output layer?
* What activation function do you want to use?
* How many inputs (features) will your network use?
* What is the appropriate loss function to use?

**Note: the only scenario where you'd use a method like this to fit a basic linear model is if your training set was too large to use other methods.  For a basic regression model there are much faster approaches to parameter estimation based on direct matrix manipulations.**  We're doing this to get practice setting up models in Keras and working with the results.  You'll likely see that when the estimation process stops the loss function is still decreasing, indicating that you have not yet reached a minimum of the negative log-likelihood.  If we really wanted the best-performing model, we'd let estimation keep running.  You can do that if you want to, but there's no need to for the purpose of this assignment.


In [16]:
# define a multinomial logistic regression model: one layer, with a sigmoid activation and 2 inputs
prices_model = models.Sequential()
prices_model.add(layers.Dense(
    1,
    activation = 'linear',
    input_shape = (36,)))

# compile the model using stochastic gradient descent for optimization,
# binary cross-entropy loss, and measuring performance by classification accuracy
prices_model.compile(
    optimizer = 'adam',
    loss = 'mean_squared_error')

# Estimate the model parameters
history = prices_model.fit(
    X_train, y_train,
    validation_data = (X_val, y_val),
    epochs = 1000,
    batch_size = X_train.shape[0])

Train on 1875 samples, validate on 469 samples
Epoch 1/1000
1875/1875 [==============================] - 0s 68us/step - loss: 37168029696.0000 - val_loss: 299924531030533030805504.0000
Epoch 2/1000
1875/1875 [==============================] - 0s 3us/step - loss: 283032591677778359746560.0000 - val_loss: inf
Epoch 3/1000
1875/1875 [==============================] - 0s 3us/step - loss: inf - val_loss: inf
Epoch 4/1000
1875/1875 [==============================] - 0s 3us/step - loss: inf - val_loss: inf
Epoch 5/1000
1875/1875 [==============================] - 0s 3us/step - loss: inf - val_loss: inf
Epoch 6/1000
1875/1875 [==============================] - 0s 2us/step - loss: inf - val_loss: nan
Epoch 7/1000
1875/1875 [==============================] - 0s 3us/step - loss: nan - val_loss: nan
Epoch 8/1000
1875/1875 [==============================] - 0s 3us/step - loss: nan - val_loss: nan
Epoch 9/1000
1875/1875 [==============================] - 0s 3us/step - loss: nan - val_loss: nan
Epoch

#### (c) Get an evaluation of the performance of your model on the test set using the built-in Keras function `evaluate`.  Is there evidence that your model has overfit the training data?  (See the solutions for lab 1 or ask if you're not sure what this means.)

In [14]:
prices_model.evaluate(X_test, y_test)
prices_model.predict

586/586 [==============================] - 0s 59us/step


38175364120.464165

#### (d) Find the test set performance of your model by manually obtaining the predicted values and computing the test set mean squared error based on them.

In [15]:
(w, b) = prices_model.layers[0].get_weights()
b = b.reshape(b.shape[0], 1)
print("b shape = " + str(b.shape))
print("w shape = " + str(w.shape))

# Calculate a vector of z's
z = b + np.dot(w.T, X_test.T)

# Calculate a vector of a's
a = z

# Calculate and print the test set MSE
np.mean((y_test - a.T)**2)

b shape = (1, 1)
w shape = (36, 1)


38175365220.432816

## Problem 2: Forest cover type prediction
We have a data set from with characteristics of land in national forests; our goal is to predict the type of forest on that land.  There are 7 possible forest types: Spruce/Fir, Lodgepole Pine, Ponderosa Pine, Cottonwood/Willow, Aspen, Douglas-fir, or Krummholz.  Our features are things like elevation, slope of land, distance to water, soil type, and so on.  (Original data source: https://archive.ics.uci.edu/ml/datasets/Covertype)



In [0]:
# read in data
forest_cover = pd.read_csv("http://www.evanlray.com/data/UCIML/forest_cover/covtype.data")

# drop a couple of columns of all 0's
to_drop = [20, 28]
forest_cover.drop(forest_cover.columns[to_drop], axis = 1, inplace=True)

# how much data do we have?
print("shape of forest_cover = " + str(forest_cover.shape))

# perform a train/validation/test split
# the indices of the split are the same as what was used in the original paper.
# note that their test set is much larger than their train set, but you would
# never do this in real life.
train_slice = slice(0, 11340)
val_slice = slice(11340, 11340+3780)
test_slice = slice(11340+3780, 11340+3780+565892)

X_train = forest_cover.iloc[train_slice, 0:52].to_numpy()
y_train = forest_cover.iloc[train_slice, 52].to_numpy() - 1

X_val = forest_cover.iloc[val_slice, 0:52].to_numpy()
y_val = forest_cover.iloc[val_slice, 52].to_numpy() - 1

X_test = forest_cover.iloc[test_slice, 0:52].to_numpy()
y_test = forest_cover.iloc[test_slice, 52].to_numpy() - 1

shape of forest_cover = (581011, 55)


#### (a) In lab 1, I claimed that an essential preprocessing step for neural networks is to center and scale the inputs.  That isn't really necessary with just a multiple regression model like in problem 1, but it is necessary as soon as you start using sigmoid or softmax activations -- which you'll need in this problem.  Do the input normalization here.

Note that you did this in lab 1, so you could just go find your old code there.  But you might like to try recreating it from scratch first.  (You want to eventually just know how to do things.)

In [0]:
X_train_mean = np.mean(X_train, axis = 0) # add a call to np.mean here.  what will you use for the axis?
X_train = X_train - X_train_mean # add code here to subtract the column means from X_train.  How will broadcasting work?
X_train_std = np.std(X_train, axis = 0) # add a call to np.std here.  what will you use for the axis?
X_train = X_train / X_train_std # add code here to divide X_train by the column standard deviations.  How will broadcasting work?

# normalize X_val, but using X_train_mean and X_train_std to do the normalization
X_val = X_val - X_train_mean  # add code here to subtract the column means from X_val
X_val = X_val / X_train_mean  # add code here to divide X_val by the column standard deviations

# normalize X_test, but using X_train_mean and X_train_std to do the normalization
X_test = X_test - X_train_mean  # add code here to subtract the column means from X_test
X_test = X_test / X_train_mean  # add code here to divide X_test by the column standard deviations

#### (b) Define an appropriate model in Keras.  Use stochastic gradient descent ('sgd') for your optimizer and 'accuracy' as an evaluation metric, and run the estimation process for 1000 epochs using a batch size equal to the number of examples in your training set.  We'll see what these things mean starting next week.
Things to consider:

* How many units do you need in your output layer?
* What activation function do you want to use?
* How many inputs (features) will your network use?
* What is the appropriate loss function to use?

In [0]:
# define a multinomial logistic regression model: one layer, with a sigmoid activation and 2 inputs
multi_logistic_model = models.Sequential()
multi_logistic_model.add(layers.Dense(
    7,
    activation = 'softmax',
    input_shape = (52,)))

# compile the model using stochastic gradient descent for optimization,
# binary cross-entropy loss, and measuring performance by classification accuracy
#sgd = optimizers.SGD(lr=0.5, momentum=0.1, nesterov=True)
multi_logistic_model.compile(
    optimizer = 'sgd',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'])

# Estimate the model parameters
history = multi_logistic_model.fit(
    X_train, y_train,
    validation_data = (X_val, y_val),
    epochs = 1000,
    batch_size = X_train.shape[0])

Train on 11340 samples, validate on 3780 samples
Epoch 1/1000
11340/11340 [==============================] - 0s 16us/step - loss: 2.7979 - acc: 0.0808 - val_loss: 6.5279 - val_acc: 0.1233
Epoch 2/1000
11340/11340 [==============================] - 0s 3us/step - loss: 2.7848 - acc: 0.0817 - val_loss: 6.4993 - val_acc: 0.1238
Epoch 3/1000
11340/11340 [==============================] - 0s 2us/step - loss: 2.7718 - acc: 0.0826 - val_loss: 6.4707 - val_acc: 0.1238
Epoch 4/1000
11340/11340 [==============================] - 0s 2us/step - loss: 2.7589 - acc: 0.0843 - val_loss: 6.4424 - val_acc: 0.1249
Epoch 5/1000
11340/11340 [==============================] - 0s 2us/step - loss: 2.7461 - acc: 0.0856 - val_loss: 6.4142 - val_acc: 0.1275
Epoch 6/1000
11340/11340 [==============================] - 0s 2us/step - loss: 2.7334 - acc: 0.0873 - val_loss: 6.3862 - val_acc: 0.1304
Epoch 7/1000
11340/11340 [==============================] - 0s 2us/step - loss: 2.7208 - acc: 0.0883 - val_loss: 6.3583 - 

#### (c) Get an evaluation of the performance of your model on the test set using the built-in Keras function `evaluate`.  Is there evidence that your model has overfit the training data?  (See the solutions for lab 1 or ask me if you're not sure what this means.)

In [0]:
multi_logistic_model.evaluate(x = X_test, y = y_test)

565891/565891 [==============================] - 10s 18us/step


[3.1103090765344215, 0.49232449358622066]

#### (d) Write your own version of the softmax function according to the docstring in the starter function below.
Note that this function expects each observation to be represented in a column of the matrix `z`.

In [0]:
def my_softmax(z):
  '''
  Calculate softmax(z) where z is a K by M matrix

  Arguments:
   - z, a K by M matrix: the entry in row j and column m of z contains 
     b_j + w_j^T x^(i), the linear combination of input features for class j and
     observation m

  Return:
   - a K by M matrix where column m is calculated as softmax of column m of z
  '''
  

If you want, you can test your function above out with the following code to make sure it seems to be working (all entries of the result should be non-negative, and the columns should add up to 1):

In [0]:
z = np.array([[17, 5, 2, 7],
              [5, 5, 1, 2],
              [2, 2, 6, -10]])
my_softmax(z)

array([[9.99993550e-01, 4.87855551e-01, 1.78679819e-02, 9.93307108e-01],
       [6.14417272e-06, 4.87855551e-01, 6.57326319e-03, 6.69285065e-03],
       [3.05900347e-07, 2.42888977e-02, 9.75558755e-01, 4.11222956e-08]])

#### (e) Find the test set performance of your model by manually obtaining the predicted values.

At some point, you will need to use the softmax function.  Although you wrote your own softmax function above, you should instead use the softmax function from the scipy package that was imported above.  Its implementation is more numerically stable; I'll talk more about this some time later.

To find the class with the highest probability based on your `a` array, check out the [argmax](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html) function in numpy.  This function will find the index of the largest entry along a given `axis` in a vector.

In [0]:
# The next two lines extract the estimated weights w and bias b from the model
# fit and then reshape b to be a 7 by 1 matrix.
(w, b) = multi_logistic_model.layers[0].get_weights()
b = b.reshape(b.shape[0], 1)

# Calculate the vector z here:
# This should involve b, X_test, and w.  Use np.dot() and broadcasting.
# Do you need to find any transposes?
z = b + np.dot(w.T, X_test.T)

# Calculate the activation vector a:
a = softmax(z, axis = 0)

# Find the predicted value y hat for each observation
y_hat = np.argmax(a, axis = 0)

# For each prediction, determine whether the prediction was correct by
# comparing it to the observed test set response.  The result of this
# calculation should be a logical vector of the same shape as y_test
# that is True for cases where the test set prediction was correct and False
# for test set cases where the prediction was wrong.
# Careful!  Make sure y_hat and y_test have the same shape before comparing
# them, or else you'll accidentally broadcast the comparison and be confused...
y_hat_correct_lgl = (y_hat.T == y_test)

# Determine what proportion of your test set predictions were correct by
# calculating the *mean* of the values in the y_hat_correct_lgl variable.
# Note that any True values (correct predictions) are converted to 1 and False
# values are converted to 0 when you do this calculation.
proportion_correct = np.mean(y_hat_correct_lgl)

# Print the proportion correct so you can see it after running this code cell
print("Proportion correct = " + str(proportion_correct))

Proportion correct = 0.49232449358622066
